In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.sparse as sps
%matplotlib inline

#train_final.csv - the training set of interactions
train_final = pd.read_csv('input/train_final.csv', delimiter = "\t");

#tracks_final.csv - supplementary information about the items
tracks_final = pd.read_csv('input/tracks_final.csv', delimiter = "\t");

#playlists_final.csv - supplementary information about the users
playlists_final = pd.read_csv('input/playlists_final.csv', delimiter = "\t");

#target_playlists.csv - the set of target playlists that will receive recommendations
target_playlists = pd.read_csv('input/target_playlists.csv');

#target_tracks.csv - the set of target items (tracks) to be recommended
target_tracks = pd.read_csv('input/target_tracks.csv');


In [2]:
#Now we want to remove some redundant stuff. 

#We will remove all songs which are not occurring more than 10 times in train_final
#Nevertheless, we still want to keep all tracks which are in the target tracks.  

popularity = train_final.groupby(by="track_id").playlist_id.nunique().to_frame()

#remove index name
popularity.reset_index(level = 0, inplace = True)

#Rename the columns
popularity.columns = ['track_id','occurrences']

#Remove all targeted tracks - TESTED, working as expected
tracks_relevant = popularity[~popularity['track_id'].isin(target_tracks['track_id'])]

#Remove tracks occurring less than 10 times
tracks_relevant = tracks_relevant[tracks_relevant['occurrences'] > 10]

#Add the targeteted tracks back again
tracks_relevant = pd.concat([tracks_relevant, target_tracks])

tracks_relevant.shape


(41756, 2)

In [ ]:
#create train_relevant

In [3]:
#Create a User Rating Matrix. In our case, User is represented by playlist
#And the ratings are implicit binary, 1/0 depending on wether a song is in the playlist. 

URM_all = sps.csr_matrix(np.zeros((playlists_final.shape[0], tracks_relevant.shape[0])))

URM_all

<57561x41756 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [4]:
#Now lets take a look at the tags.
tracks_final.head()

,track_id,artist_id,duration,playcount,album,tags
0,2972914,144,224000,49.0,[7],"[54087, 1757, 1718, 116712, 189631]"
1,2750239,246,157000,1.0,[8],"[189631, 3424, 177424, 46208, 205245]"
2,1550729,144,217000,554.0,[9],"[54087, 109806, 46869, 183258, 54337]"
3,2169950,144,207000,200.0,[9],"[54087, 70618, 207003, 109806, 116712]"
4,1903709,144,198000,5.0,[None],"[54087, 81223, 116712, 215342, 71028]"


In [5]:
#Lets translate the tags to indexes. 
tracks_final['tags'].head()


tags_indexes = {}
counter = 0; #We will start at 1, reserving col 0 for indexes. 
for row in tracks_final['tags']:
    tags = row.strip('[ ]').split(', ')
    for tag in tags:
        if len(tag) > 0: 
            tag = int(tag)
            if not(tag in tags_indexes):
                tags_indexes[tag] = counter #Lets make it int to make it easier
                counter += 1;

print(len(tags_indexes))

31900


In [6]:
#If we translate each track_id to a track_index which will serve as matrix index, we can save a lot of time. 


#We need a way to get from track_id to index in O(1).
#Let's create a dictionary

tracks_indexes = {}
index_to_track_id = {}
counter = 0; #We will start at 1, reserving col 0 for indexes.  
for track_id in tracks_final['track_id']:
    tracks_indexes[track_id] = counter
    index_to_track_id[counter] = int(track_id)
    counter += 1;
    
    
print("We have {} unique tracks with {} unique tags".format(len(tracks_indexes), len(tags_indexes)))


We have 100000 unique tracks with 31900 unique tags


In [7]:
#and a way to get from playlist_id to index in O(1)


playlist_indexes = {}
index_to_playlist = {}
counter = 0; 
for playlist_id in playlists_final['playlist_id']:
    playlist_indexes[playlist_id] = counter
    counter += 1;


In [8]:
#Now we can create an Item Content Matrix. 

ICM_all = np.zeros((len(tracks_indexes), len(tags_indexes)), int)
print(ICM_all.shape)



(100000, 31900)


In [9]:
#tracks_relevant.reset_index(level = 0, inplace = True)

tracks = pd.merge(tracks_relevant, tracks_final, how='inner', on='track_id')

print(tracks.head())

tracks = tracks.as_matrix()

   occurrences  track_id  artist_id  duration  playcount     album  \
0         18.0       360     169649    194000      522.0   [77662]   
1         11.0      1376     381303    270000      629.0  [180587]   
2         12.0      2623      36019    329000     7081.0   [17295]   
3         15.0      2891     310373    194000      196.0  [142650]   
4         14.0      2901     163720    293000      322.0   [74384]   

                                      tags  
0      [70618, 70251, 70625, 25307, 11056]  
1  [205245, 254105, 11056, 209598, 189631]  
2    [122769, 23214, 48976, 117167, 90254]  
3     [189631, 54087, 70618, 94794, 61837]  
4     [193464, 205245, 46208, 92324, 3982]  


In [10]:

#So let's fill it with our data.


for row in tracks: 
    track_id = row[1]
    tags = row[6].strip('[ ]').split(', ')
    for tag in tags:
        if len(tag) > 0:
            tag_index = tags_indexes[int(tag)]
            track_index = tracks_indexes[track_id]
            ICM_all[track_index,tag_index] = 1
            #print("Added tag {} to track {}".format(tag, track_id))
            

print(ICM_all)

[[1 1 1 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [1 0 0 ..., 0 0 0]
 ..., 
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]
 [0 0 0 ..., 0 0 0]]


In [11]:
ICM_all_sparse = sps.csr_matrix(ICM_all)
ICM_all_sparse

<100000x31900 sparse matrix of type '<class 'numpy.int64'>'
	with 203506 stored elements in Compressed Sparse Row format>

In [ ]:
#Lets take a look at the data. 

features_per_item = (ICM_all_sparse > 0).sum(axis=1)
items_per_feature = (ICM_all_sparse > 0).sum(axis=0)

features_per_item = np.sort(features_per_item)
items_per_feature = np.sort(items_per_feature)

print(features_per_item.shape)
print(items_per_feature.shape)

In [12]:
train_final.shape

(1040522, 2)

In [ ]:
#Lets build that URM matrix. Should be nofplaylists x nofitems

item_playlist_matrix = np.zeros([playlists_final.shape[0], tracks_final.shape[0]],int) 

interactions = train_final.as_matrix()
for row in interactions:
    #Lets get the info
    playlist_id = row[0]
    track_id = row[1]
    
    #Now lets get the proper indexes. 
    playlist_index = playlist_indexes[playlist_id]
    track_index = tracks_indexes[track_id]
    
    #And now lets add it to the matrix
    item_playlist_matrix[playlist_index][track_index] = 1
    

print(item_playlist_matrix.shape)
print("hej1")

In [13]:
#Let's Split the Data and create evluation functions. 

train_test_split = 0.80

numInteractions = train_final.shape[0]


train_mask = np.random.choice([True,False], numInteractions, [train_test_split, 1-train_test_split])

playlistList = np.zeros((train_final['playlist_id'].shape[0], 1), int)
playlistList[:, 0] = train_final['playlist_id']

#itemList = np.zeros((train_final['track_id'].shape[0], 1), int)
#itemList[:, 0] = train_final['track_id']
playlistList = train_final['playlist_id'].values
itemList = train_final['track_id'].values

#Translate ids
playlistList_translated = np.zeros(playlistList.shape)
itemList_translated = np.zeros(itemList.shape)
for i in range(train_final.shape[0]):
    playlistList_translated[i] = playlist_indexes[playlistList[i]]
    itemList_translated[i] = tracks_indexes[itemList[i]]



ratingList = np.ones((playlistList.shape), int)


URM_train = sps.coo_matrix((ratingList[train_mask], (playlistList_translated[train_mask], itemList_translated[train_mask])))
URM_train = URM_train.tocsr()

print(URM_train.shape)
print(playlistList.shape)
print(itemList.shape)




test_mask = np.logical_not(train_mask)

URM_test = sps.coo_matrix((ratingList[test_mask], (playlistList[test_mask], itemList[test_mask])))
URM_test = URM_test.tocsr()

def precision(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    precision_score = np.sum(is_relevant, dtype=np.float32) / len(is_relevant)
    
    return precision_score

def recall(recommended_items, relevant_items):
    
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    recall_score = np.sum(is_relevant, dtype=np.float32) / relevant_items.shape[0]
    
    return recall_score

def MAP(recommended_items, relevant_items):
   
    is_relevant = np.in1d(recommended_items, relevant_items, assume_unique=True)
    
    # Cumulative sum: precision at 1, at 2, at 3 ...
    p_at_k = is_relevant * np.cumsum(is_relevant, dtype=np.float32) / (1 + np.arange(is_relevant.shape[0]))
    
    map_score = np.sum(p_at_k) / np.min([relevant_items.shape[0], is_relevant.shape[0]])

    return map_score

def evaluate_algorithm(URM_test, recommender_object, at=5):
    
    cumulative_precision = 0.0
    cumulative_recall = 0.0
    cumulative_MAP = 0.0
    
    num_eval = 0


    for i,user_id in  enumerate(userList_unique):
        
        if i % 500 == 0:
            print("User %d of %d" % (i, len(userList_unique)))

        relevant_items = URM_test[user_id].indices
        
        if len(relevant_items)>0:
            
            recommended_items = recommender_object.recommend(user_id, at=at)
            num_eval+=1

            cumulative_precision += precision(recommended_items, relevant_items)
            cumulative_recall += recall(recommended_items, relevant_items)
            cumulative_MAP += MAP(recommended_items, relevant_items)


    cumulative_precision /= num_eval
    cumulative_recall /= num_eval
    cumulative_MAP /= num_eval
    
    print("Recommender performance is: Precision = {:.4f}, Recall = {:.4f}, MAP = {:.4f}".format(
        cumulative_precision, cumulative_recall, cumulative_MAP))



(57560, 100000)
(1040522,)
(1040522,)


In [14]:
class BasicItemKNNRecommender(object):
    """ ItemKNN recommender with cosine similarity and no shrinkage"""

    def __init__(self, URM, k=50, shrinkage=100, similarity='cosine'):
        self.dataset = URM
        self.k = k
        self.shrinkage = shrinkage
        self.similarity_name = similarity
        if similarity == 'cosine':
            self.distance = Cosine(shrinkage=self.shrinkage)
        elif similarity == 'pearson':
            self.distance = Pearson(shrinkage=self.shrinkage)
        elif similarity == 'adj-cosine':
            self.distance = AdjustedCosine(shrinkage=self.shrinkage)
        else:
            raise NotImplementedError('Distance {} not implemented'.format(similarity))

    def __str__(self):
        return "ItemKNN(similarity={},k={},shrinkage={})".format(
            self.similarity_name, self.k, self.shrinkage)

    def fit(self, X):
        item_weights = self.distance.compute(X)
        
        item_weights = check_matrix(item_weights, 'csr') # nearly 10 times faster
        print("Converted to csr")
        
        # for each column, keep only the top-k scored items
        # THIS IS THE SLOW PART, FIND A BETTER SOLUTION        
        values, rows, cols = [], [], []
        nitems = self.dataset.shape[1]
        for i in range(nitems):
            if (i % 10000 == 0):
                print("Item %d of %d" % (i, nitems))
                
            this_item_weights = item_weights[i,:].toarray()[0]
            top_k_idx = np.argsort(this_item_weights) [-self.k:]
                        
            values.extend(this_item_weights[top_k_idx])
            rows.extend(np.arange(nitems)[top_k_idx])
            cols.extend(np.ones(self.k) * i)
        self.W_sparse = sps.csc_matrix((values, (rows, cols)), shape=(nitems, nitems), dtype=np.float32)

    def recommend(self, user_id, at=None, exclude_seen=True):
        # compute the scores using the dot product
        user_profile = self.dataset[user_id]
        scores = user_profile.dot(self.W_sparse).toarray().ravel()

        # rank items
        ranking = scores.argsort()[::-1]
        if exclude_seen:
            ranking = self._filter_seen(user_id, ranking)
            
        export = [0,0,0,0,0]
        for i in range(5):
            t_id = index_to_track_id[ranking[i]]
            export[i] = t_id
            
        return export
    def _filter_seen(self, user_id, ranking):
        user_profile = self.dataset[user_id]
        seen = user_profile.indices
        unseen_mask = np.in1d(ranking, seen, assume_unique=True, invert=True)
        return ranking[unseen_mask]
    
print("done")

done


In [15]:
def check_matrix(X, format='csc', dtype=np.float32):
    if format == 'csc' and not isinstance(X, sps.csc_matrix):
        return X.tocsc().astype(dtype)
    elif format == 'csr' and not isinstance(X, sps.csr_matrix):
        return X.tocsr().astype(dtype)
    elif format == 'coo' and not isinstance(X, sps.coo_matrix):
        return X.tocoo().astype(dtype)
    elif format == 'dok' and not isinstance(X, sps.dok_matrix):
        return X.todok().astype(dtype)
    elif format == 'bsr' and not isinstance(X, sps.bsr_matrix):
        return X.tobsr().astype(dtype)
    elif format == 'dia' and not isinstance(X, sps.dia_matrix):
        return X.todia().astype(dtype)
    elif format == 'lil' and not isinstance(X, sps.lil_matrix):
        return X.tolil().astype(dtype)
    else:
        return X.astype(dtype)

In [16]:
import scipy
class ISimilarity(object):
    """Abstract interface for the similarity metrics"""

    def __init__(self, shrinkage=10):
        self.shrinkage = shrinkage

    def compute(self, X):
        pass


class Cosine(ISimilarity):
    def compute(self, X):
        # convert to csc matrix for faster column-wise operations
        X = check_matrix(X, 'csc', dtype=np.float32)

        # 1) normalize the columns in X
        # compute the column-wise norm
        # NOTE: this is slightly inefficient. We must copy X to compute the column norms.
        # A faster solution is to  normalize the matrix inplace with a Cython function.
        Xsq = X.copy()
        Xsq.data **= 2
        norm = np.sqrt(Xsq.sum(axis=0))
        norm = np.asarray(norm).ravel()
        norm += 1e-6
        # compute the number of non-zeros in each column
        # NOTE: this works only if X is instance of sparse.csc_matrix
        col_nnz = np.diff(X.indptr)
        # then normalize the values in each column
        X.data /= np.repeat(norm, col_nnz)
        print("Normalized")

        # 2) compute the cosine similarity using the dot-product
        dist = X * X.T
        print("Computed")
        
        # zero out diagonal values
        dist = dist - sps.dia_matrix((dist.diagonal()[scipy.newaxis, :], [0]), shape=dist.shape)
        print("Removed diagonal")
        
        # and apply the shrinkage
        if self.shrinkage > 0:
            dist = self.apply_shrinkage(X, dist)
            print("Applied shrinkage")    
        
        return dist

    def apply_shrinkage(self, X, dist):
        # create an "indicator" version of X (i.e. replace values in X with ones)
        X_ind = X.copy()
        X_ind.data = np.ones_like(X_ind.data)
        # compute the co-rated counts
        co_counts = X_ind * X_ind.T
        # remove the diagonal
        co_counts = co_counts - sps.dia_matrix((co_counts.diagonal()[scipy.newaxis, :], [0]), shape=co_counts.shape)
        # compute the shrinkage factor as co_counts_ij / (co_counts_ij + shrinkage)
        # then multiply dist with it
        co_counts_shrink = co_counts.copy()
        co_counts_shrink.data += self.shrinkage
        co_counts.data /= co_counts_shrink.data
        dist.data *= co_counts.data
        return dist


In [17]:
URM_train.shape

(57560, 100000)

In [23]:
rec = BasicItemKNNRecommender(URM=URM_train, shrinkage=0.0, k=50)
rec.fit(ICM_all_sparse)
print("done")

Normalized
Computed
Removed diagonal
Converted to csr
Item 0 of 100000
Item 10000 of 100000
Item 20000 of 100000
Item 30000 of 100000
Item 40000 of 100000
Item 50000 of 100000
Item 60000 of 100000
Item 70000 of 100000
Item 80000 of 100000
Item 90000 of 100000
done


In [25]:
ICM_all = ICM_all_sparse

In [26]:
num_tot_items = ICM_all.shape[0]

# let's count how many items have a certain feature
items_per_feature = (ICM_all > 0).sum(axis=0)

IDF = np.array(np.log(num_tot_items / items_per_feature))[0]

print(ICM_all.shape)
print(IDF.shape)

(100000, 31900)
(31900,)


/Users/philipclaesson/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in true_divide
  


In [34]:
ICM_idf = sps.csr_matrix(ICM_all, dtype=np.float64)
# compute the number of non-zeros in each col
# NOTE: this works only if X is instance of sparse.csc_matrix
col_nnz = np.diff(check_matrix(ICM_idf, 'csc').indptr)
print(col_nnz.shape)
print(ICM_idf.shape)
print(IDF.shape)
# then normalize the values in each col
ICM_idf.data *= np.repeat(IDF, col_nnz)

(31900,)
float64
float64


In [36]:
rec_idf = BasicItemKNNRecommender(URM=URM_train, shrinkage=0.0, k=50)
rec_idf.fit(ICM_idf)

Normalized
Computed
Removed diagonal
Converted to csr
Item 0 of 100000
Item 10000 of 100000
Item 20000 of 100000
Item 30000 of 100000
Item 40000 of 100000
Item 50000 of 100000
Item 60000 of 100000
Item 70000 of 100000
Item 80000 of 100000
Item 90000 of 100000


In [37]:
zeros = np.zeros((target_playlists.size, 6), dtype = int)
recommendations = pd.DataFrame(zeros)
recommendations.columns = ['playlist_id', 1, 2, 3, 4, 5]

counter = 0
for playlist_id in target_playlists['playlist_id']:
    
    if counter % 1000 == 0: 
        print ("%s out of 10000 playlists" %(counter))
    
    
    playlist_id_translated = playlist_indexes[int(playlist_id)]
    #print(rec.recommend(playlist_id_translated, at=5))
    recommendations.iloc[counter, 1:6] = rec.recommend(playlist_id_translated, at=5)
    recommendations.iloc[counter, 0] = playlist_id
    counter += 1
    
#print(recommendations)
print("done")

0 out of 10000 playlisys
1000 out of 10000 playlisys
2000 out of 10000 playlisys
3000 out of 10000 playlisys
4000 out of 10000 playlisys
5000 out of 10000 playlisys
6000 out of 10000 playlisys
7000 out of 10000 playlisys
8000 out of 10000 playlisys
9000 out of 10000 playlisys
done


In [38]:
def save_to_file():
    #Saves the recommendations dataframe to the .csv-file. 
    np.savetxt("recommendations_idf.csv",recommendations, fmt = '%s,%s %s %s %s %s', header = "playlist_id,track_ids", newline = "\n")
    
    
def test():
    #Do something
    print("Result: ")
    pass


save_to_file()
print(recommendations.head)

<bound method NDFrame.head of       playlist_id        1        2        3        4        5
0        10024884  3156505  1469927  3063256   848214    60264
1        10624787   194963  2319873  4924281   497324  1818963
2         4891851   557629  4735764  2526786  2940539  1809338
3         4267369  1154985  3137640   139177  1942809  3658983
4           65078   509377   119305  2863566  1742595  2386315
5        10637124   346386  2340644  2123901  2740526  1633063
6         3223162   153334  1820710   498087  1438727  2874676
7         7541503  1295068    76828  3738541  2456972  2086779
8         6189367  2019507  1054050  1195554  3051311  3869194
9         8459943  3536664  1003537  2485522  3129861   826298
10       10138804  1388061  2744692  1229845   468510  2515331
11       10562075  3708968   683230  1212246  3593525  1881344
12       10184821  3194368  1253640  2426923  2513863   819539
13        4189678  1175499   511452  2944080   253512  1796061
14        6299524  174441